<a href="https://colab.research.google.com/github/sljm12/machine_learning_notebooks/blob/master/GPT2_for_Data_Augementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Want to investigate using GPT2 for Data Augenmentation

https://huggingface.co/blog/how-to-generate

https://arxiv.org/pdf/2003.02245v1.pdf

https://github.com/falloutdurham/beginners-pytorch-deep-learning/blob/master/chapter9/Chapter9.5.ipynb

https://docs.fast.ai/tutorial.transformers

In [1]:
!pip -qq install transformers

In [1]:
import torch
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained('gpt2',num_beams=4)

input_ids = tokenizer.encode("The night is still young, as the world sleeps ",return_tensors='pt')


generated = model.generate(input_ids, max_length=100, num_beams=5, early_stopping=True, no_repeat_ngram_size=2, num_return_sequences=5, top_k=0, top_p=0.9)

for i in generated:
  sequence = tokenizer.decode(i, skip_special_tokens=True)
  print(sequence)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The night is still young, as the world sleeps  and there is no sign of the sun rising. The sun is rising, and the moon is falling. There is nothing to be seen, but the sky is full of stars.
The sun has risen. It is time for us to go back to sleep. We are going to wake up in the morning and we will be able to look at the stars and see what is going on around us. This is the time we need to
The night is still young, as the world sleeps  and there is no sign of the sun rising. The sun is rising, and the moon is falling. There is nothing to be seen, but the sky is full of stars.
The sun has risen. It is time for us to go back to sleep. We are going to wake up in the morning and we will be able to look at the stars and see what is going on around us. This is the time when we should
The night is still young, as the world sleeps  and there is no sign of the sun rising. The sun is rising, and the moon is falling. There is nothing to be seen, but the sky is full of stars.
The sun has risen. It 

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained('gpt2',num_beams=4)

generated = tokenizer.encode("Stephen Lee has two kids Chloe and Caleb.")
context = torch.tensor([generated])
past = None

for i in range(100):
    output, past = model(context, past=past)
    token = torch.argmax(output[..., -1, :])

    generated += [token.tolist()]
    context = token.unsqueeze(0)

sequence = tokenizer.decode(generated)

print(sequence)

# Finetuning

## Dataset

In [3]:
!wget -qq https://www.dropbox.com/s/duoi46s4db28xac/news-category-dataset.zip
!unzip news-category-dataset.zip

Archive:  news-category-dataset.zip
replace News_Category_Dataset_v2.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [2]:
import json
from  pathlib import Path
data = []
with open("/content/News_Category_Dataset_v2.json") as f:
  lines = f.readlines()
  for l in lines:
    j=json.loads(l)
    data.append(j)

In [3]:
df=pd.DataFrame(data=data)

In [4]:
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [5]:
df["category"].unique()

array(['CRIME', 'ENTERTAINMENT', 'WORLD NEWS', 'IMPACT', 'POLITICS',
       'WEIRD NEWS', 'BLACK VOICES', 'WOMEN', 'COMEDY', 'QUEER VOICES',
       'SPORTS', 'BUSINESS', 'TRAVEL', 'MEDIA', 'TECH', 'RELIGION',
       'SCIENCE', 'LATINO VOICES', 'EDUCATION', 'COLLEGE', 'PARENTS',
       'ARTS & CULTURE', 'STYLE', 'GREEN', 'TASTE', 'HEALTHY LIVING',
       'THE WORLDPOST', 'GOOD NEWS', 'WORLDPOST', 'FIFTY', 'ARTS',
       'WELLNESS', 'PARENTING', 'HOME & LIVING', 'STYLE & BEAUTY',
       'DIVORCE', 'WEDDINGS', 'FOOD & DRINK', 'MONEY', 'ENVIRONMENT',
       'CULTURE & ARTS'], dtype=object)

In [7]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained('gpt2',num_beams=4)

In [6]:
class NewsDataSet(Dataset):
    
    def __init__(self, dataframe, control_code, truncate=False, gpt2_type="gpt2", max_length=768):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.tweets = []

        # This uses the same CSV of Sentiment140 that we created in Chapter 5
        
        for i in dataframe.itertuples():
              self.tweets.append(torch.tensor(
                  self.tokenizer.encode(f"<|{control_code}|>{i[1]}<|sep|>{i[2]}<|endoftext|>")
              ))
                
        if truncate:
            self.tweets = self.tweets[:20000]
        self.tweet_count = len(self.tweets)
        
    def __len__(self):
        return self.tweet_count

    def __getitem__(self, item):
        return self.tweets[item]

In [7]:
dataset = NewsDataSet(df, 'newsheader')

In [54]:
dataset.__getitem__(1)

tensor([   27,    91, 10827, 25677,    91,    29,  8743,  4176,  5302,  1040,
         6031,   489,    78,   843,  8047,    88,  9986,  1114,   383,  2864,
         2159,  5454,   338, 15934, 10940, 50256])

In [55]:
dataset.__len__()

200853

In [32]:
tokenizer.decode(dataset.__getitem__(1))

"<|newsheader|>ENTERTAINMENT<|sep|>Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song<|endoftext|>"

In [8]:
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [9]:
def train(
    dataset,
    model,
    tokenizer,
    batch_size=16,
    epochs=4,
    lr=2e-5,
    max_seq_len=400,
    warmup_steps=5000,
    gpt2_type="gpt2",
    device="cuda",
    output_dir=".",
    output_prefix="wreckgar",
    test_mode=False,
    save_model_on_epoch=False,
):

    acc_steps = 100

    model = model.to(device)
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

In [14]:
!mkdir "/content/trained_models"

In [10]:
model = train(
    dataset,
    GPT2LMHeadModel.from_pretrained("gpt2"),
    GPT2Tokenizer.from_pretrained("gpt2"),
    batch_size=16,
    epochs=3,
    lr=3e-5,
    max_seq_len=140,
    warmup_steps=5000,
    gpt2_type="gpt2",
    device="cuda",
    output_dir="/content/trained_models",
    output_prefix="gpt_newsheadlines_categories",
    save_model_on_epoch=True
)

0it [00:00, ?it/s]

Training epoch 0


200853it [40:33, 82.53it/s]
0it [00:00, ?it/s]

Training epoch 1


200853it [40:38, 82.37it/s]
0it [00:00, ?it/s]

Training epoch 2


200853it [40:39, 82.32it/s]


In [ ]:
nmodel.save

In [12]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=100,
    top_p=0.8,
    temperature=1.,
):

    model.eval()

    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False

            generated = torch.tensor(tokenizer.encode(prompt)).to("cuda").unsqueeze(0)

            # Using top-p (nucleus sampling): https://github.com/huggingface/transformers/blob/master/examples/run_generation.py

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(
                    F.softmax(sorted_logits, dim=-1), dim=-1
                )

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.to("cpu").squeeze().numpy())
                    output_text = tokenizer.decode(output_list)

                    generated_list.append(output_text)
                    break
            
            if not entry_finished:
                output_list = list(generated.to("cpu").squeeze().numpy())
                output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
                generated_list.append(output_text)
                
    return generated_list

In [13]:
generated_tweets = generate(model.to('cuda'), GPT2Tokenizer.from_pretrained("gpt2"),"<|newsheadlines|>POLITICS<|sep|>",entry_count=10)

100%|██████████| 10/10 [00:02<00:00,  3.46it/s]


In [14]:
for i in generated_tweets:
  print(i)

<|newsheadlines|>POLITICS<|sep|>Speaker Paul Ryan Makes Retirement With $10 Million In Cash And Student Loans<|endoftext|>
<|newsheadlines|>POLITICS<|sep|>Anti-Semitic Twitter Posts At London Airport Linked To Orlando Shooting<|endoftext|>
<|newsheadlines|>POLITICS<|sep|>Clinton Campaign Donates $150,000 to DOJ To Combat 'Deadly' Anti-Immigrant Effort<|endoftext|>
<|newsheadlines|>POLITICS<|sep|>Hillary Clinton Is From A Meeting With A Young Girl<|endoftext|>
<|newsheadlines|>POLITICS<|sep|>Sen. Kirsten Gillibrand Offers Help For Gay Trump's Trans Rights<|endoftext|>
<|newsheadlines|>POLITICS<|sep|>Division Between OPPOSITE and WELLNESS CONNECTING</|sep|>Why Some Americans Think Hillary Clinton's 'Insecure' Email Address Will Be Bad For Hillary Clinton<|endoftext|>
<|newsheadlines|>POLITICS<|sep|>Russia Posters Are Done For Again: New 'House Of Cards' Trailer Looks Reasonable<|endoftext|>
<|newsheadlines|>POLITICS<|sep|>Politicians Who Won The Presidential Primary: Donald Trump, Mitt R

In [15]:
generated_tweets = generate(model.to('cuda'), GPT2Tokenizer.from_pretrained("gpt2"),"<|newsheadlines|>CRIME<|sep|>",entry_count=10)
for i in generated_tweets:
  print(i)

100%|██████████| 10/10 [00:02<00:00,  4.93it/s]

<|newsheadlines|>CRIME<|sep|>FBI Warns FBI Offbeds Moving Kids From Hidalgo County, Texas<|endoftext|>
<|newsheadlines|>CRIME<|sep|>Tears well up on U.S. News '' Investigators<|endoftext|>
<|newsheadlines|>CRIME<|sep|>Why This Disease Is So Powerful For Human Rights<|endoftext|>
<|newsheadlines|>CRIME<|sep|>2 People Found Dead in Hotel Heist After Hostage Style Attack<|endoftext|>
<|newsheadlines|>CRIME<|sep|>North Carolina Boy Shot In Pinellas To Kill Woman<|endoftext|>
<|newsheadlines|>CRIME<|sep|>Man Arrested On The Prom Back Of Cocaine, Meth, Heroin As Heroin In Dank Cages<|endoftext|>
<|newsheadlines|>CRIME<|sep|>Cop Kills Son With Teenage Shootout At Heist 'God Bless America' Posters<|endoftext|>
<|newsheadlines|>CRIME<|sep|>Philadelphia Allegedly Shot Two People Just Before Killing (VIDEO)<|endoftext|>
<|newsheadlines|>CRIME<|sep|>Homicides Reportedly Found In New York's Pregnant Is An Infraction Of The NYPD<|endoftext|>
<|newsheadlines|>CRIME<|sep|>Gunman Shot And Died At Pulse

In [16]:
generated_tweets = generate(model.to('cuda'), GPT2Tokenizer.from_pretrained("gpt2"),"<|newsheadlines|>SCIENCE<|sep|>",entry_count=10)
for i in generated_tweets:
  print(i)

100%|██████████| 10/10 [00:02<00:00,  4.88it/s]

<|newsheadlines|>SCIENCE<|sep|>It's an Open Letter to One Girl's Dad Who Lost His Job To Cancer<|endoftext|>
<|newsheadlines|>SCIENCE<|sep|>Scientific Explanation: Engineers Did It<|endoftext|>
<|newsheadlines|>SCIENCE<|sep|>Plants Could Actually Make A Virus, Study Shows<|endoftext|>
<|newsheadlines|>SCIENCE<|sep|>Where Did the Supermassive Black Hole Exist?<|endoftext|>
<|newsheadlines|>SCIENCE<|sep|>Anomaly Could Be Found A Day After Living In The World's Most Lethal Superfund Site<|endoftext|>
<|newsheadlines|>SCIENCE<|sep|>An Astronaut's Complete Guide To Jupiter's New Slope, And The Fight Against Planet Earth<|endoftext|>
<|newsheadlines|>SCIENCE<|sep|>A Cell-Robust Plan To The Rescue Your Heart<|endoftext|>
<|newsheadlines|>SCIENCE<|sep|>Antarctic Climate Change Paves Way for Huge Dark Ocean On Pole<|endoftext|>
<|newsheadlines|>SCIENCE<|sep|>'Hunting In the Shadows': An Exploration of a Shaping Inner World<|endoftext|>
<|newsheadlines|>SCIENCE<|sep|>New Marist Medical Center, C

In [17]:
generated_tweets = generate(model.to('cuda'), GPT2Tokenizer.from_pretrained("gpt2"),"<|newsheadlines|>WOLRD NEWS<|sep|>",entry_count=10)
for i in generated_tweets:
  print(i)

100%|██████████| 10/10 [00:01<00:00,  5.37it/s]

<|newsheadlines|>WOLRD NEWS<|sep|>J.D. Salinger's Native American Run Needs To Be 'Actively Blocking' The Slut Walk<|endoftext|>
<|newsheadlines|>WOLRD NEWS<|sep|>Why I Would Absolutely Leave Politics In A Lazy Place<|endoftext|>
<|newsheadlines|>WOLRD NEWS<|sep|>The Bay, Like Rivers Of The Sea<|endoftext|>
<|newsheadlines|>WOLRD NEWS<|sep|>The Chaos Strikes<|endoftext|>
<|newsheadlines|>WOLRD NEWS<|sep|>Johnnie Walker Poses A Big Price In Kratom Claim<|endoftext|>
<|newsheadlines|>WOLRD NEWS<|sep|>Obama Spokeswoman And Republicans Are Sliding Over' Bashing On Trump<|endoftext|>
<|newsheadlines|>WOLRD NEWS<|sep|>Alarming Photos Show Turtles Swimming In Water Of Tongue and Upright Nails<|endoftext|>
<|newsheadlines|>WOLRD NEWS<|sep|>There's Something About When It Happens<|endoftext|>
<|newsheadlines|>WOLRD NEWS<|sep|>Why A Girl Calls Him 'Bitch Boy' And Says 'Man In White'<|endoftext|>
<|newsheadlines|>WOLRD NEWS<|sep|>Thousands Prompts First Big Move Against HIV/AIDS Accident<|endofte

In [18]:
generated_tweets = generate(model.to('cuda'), GPT2Tokenizer.from_pretrained("gpt2"),"<|newsheadlines|>PARENTING<|sep|>",entry_count=10)
for i in generated_tweets:
  print(i)

100%|██████████| 10/10 [00:02<00:00,  4.93it/s]

<|newsheadlines|>PARENTING<|sep|>Meeting Church Leaders Who Are Taking Kids to Gospel Sites<|endoftext|>
<|newsheadlines|>PARENTING<|sep|>The Path Ahead for Kids Who Are Growing Up In School Separated From Our Parents' Stories<|endoftext|>
<|newsheadlines|>PARENTING<|sep|>5 Things You Need To Know About Making Parenting a Reality<|endoftext|>
<|newsheadlines|>PARENTING<|sep|>When I Was Loving Grace, Someone Said, 'Go Away'<|endoftext|>
<|newsheadlines|>PARENTING<|sep|>9 Things That You Can Learn From Parenting<|endoftext|>
<|newsheadlines|>PARENTING<|sep|>10 Books That Can Make You Grow Up<|endoftext|>
<|newsheadlines|>PARENTING<|sep|>This Kid Doesn't Care If He's A Star Of 'This Country'<|endoftext|>
<|newsheadlines|>PARENTING<|sep|>10 Things You Must Know About Pregnancy and Toddlers<|endoftext|>
<|newsheadlines|>PARENTING<|sep|>Parents Expect Their Kids To Call Out For Fears Of Fears Of Autism<|endoftext|>
<|newsheadlines|>PARENTING<|sep|>Tim Herring's Dad Fails To Raise Kids He Did

In [14]:
!ls -lh /content/trained_models

total 487M
-rw-r--r-- 1 root root 487M Nov 24 06:33 newsheadlines-0.pt


In [11]:
!cp /content/trained_models/gpt_newsheadlines_categories-2.pt '/content/drive/MyDrive/Machine Learning/'

In [20]:
torch.save(model,"/content/gpt_news.pt")

In [21]:
!ls -lh /content

total 618M
drwx------ 4 root root 4.0K Nov 24 06:52 drive
-rw-r--r-- 1 root root 487M Nov 24 07:18 gpt_news.pt
-rw-r--r-- 1 root root  81M Oct  1  2019 News_Category_Dataset_v2.json
-rw-r--r-- 1 root root  26M Nov 24 05:19 news-category-dataset.zip
-rw-r--r-- 1 root root  26M Nov 24 06:00 news-category-dataset.zip.1
drwxr-xr-x 1 root root 4.0K Nov 13 17:33 sample_data
drwxr-xr-x 2 root root 4.0K Nov 24 06:33 trained_models


In [23]:
torch.load()

TypeError: ignored